## Parameters

In [1]:
EXP_RESULT_DIR = "../../tmp/exps-musique-val-mini/"

In [2]:
# DEVICE_CONFIGS = ["1", "2", "3"]
# DEVICE_CONFIGS = ["0,1", "2,3"]
# DEVICE_CONFIGS = ["0,1,2,3"]
DEVICE_CONFIGS = ["1"]

In [3]:
common_params = {
    "params.model.path": [
        # Small models
        # "Qwen/Qwen2.5-1.5B-Instruct",
        # "Qwen/Qwen2.5-7B-Instruct",
        "meta-llama/Llama-3.1-8B-Instruct",
        
        # Medium models
        # "Qwen/Qwen2.5-14B-Instruct",
        # "Qwen/Qwen2.5-32B-Instruct",
        # "Qwen/Qwen2.5-Coder-32B-Instruct",
        # "meta-llama/Llama-3.1-70B-Instruct",
        # "Qwen/QwQ-32B",
        
        # GRPO models
        # "bdsaglam/Qwen2.5-1.5B-Instruct-ragent-musique",

        "bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-musique-merged",  # Ragent-1
        # "/home/baris/repos/verifiers/outputs/Llama-3.1-8B-Instruct-ragent-20250421_000014-400", # Ragent-2
        # "/home/baris/repos/verifiers/outputs/Llama-3.1-8B-Instruct-ragent-20250421_000014-1600", # Ragent-2
        # "/home/baris/repos/verifiers/outputs/Llama-3.1-8B-Instruct-ragent-grpo-20250427_095331-merged",  # Ragent-3
        # "/home/baris/repos/verifiers/outputs/Llama-3.1-8B-Instruct-ragent-grpo-20250508_213215-merged",  # Ragent-4
        "/home/baris/repos/verifiers/outputs/Llama-3.1-8B-Instruct-ragent-grpo-20250520_080809-merged",  # Ragent-5
    ],
    "params.model.temperature": [
        # 0.3,
        0.5,
        # 0.7,
    ],
    "params.model.top_p": [
        0.95,
    ],
    "params.model.few_shot_prob": [
        0.0,
        # 1.0,
    ],
    "params.retriever.name": [
        # "semantic",
        # "lexical",
        # "hybrid",
        "hybrid-tei",
    ],
    "params.retriever.top_k": [
        1,
        # 2,
        # 3,
    ],
    "params.repeat": [
        1,
        5,
    ],
    "params.run": [
        1,
        # 2,
        # 3
    ],
}

In [4]:
varying_params_list = [
    {
        "params.dataset.path": ["bdsaglam/musique-mini"],
        "params.dataset.name": ["answerable"],
        "params.dataset.split": ["validation"],
    },
    # {
    #     "params.dataset.path": ["bdsaglam/hotpotqa-distractor"],
    #     "params.dataset.name": ["default"],
    #     "params.dataset.split": ['validation[:300]'],
    # },
]

## Setup

In [5]:
# AUTOGENERATED! DO NOT EDIT! File to edit: ../../nbs/dvc.experiment.ipynb.

# %% auto 0
__all__ = ["parse_params", "parse_metrics", "parse_experiment", "parse_experiments", "load_experiments"]

# %% ../../nbs/dvc.experiment.ipynb 3
import json
from typing import Generator


# %% ../../nbs/dvc.experiment.ipynb 4
def parse_params(record):
    params_node = record.get("data", {}).get("params", {})
    params = {}
    for k, v in params_node.items():
        params.update(v.get("data", {}))
    return params


def parse_metrics(record):
    metrics_node = record.get("data", {}).get("metrics", {})
    metrics = {}
    for k, v in metrics_node.items():
        metrics.update(v.get("data", {}))
    return metrics


def parse_experiment(record):
    return {
        "id": record["rev"],
        "name": record["name"],
        "params": parse_params(record),
        "metrics": parse_metrics(record),
    }


def parse_experiments(data: list[dict]) -> Generator[dict, None, None]:
    for node in data:
        if node.get("error"):
            continue
        commit = node.get("rev")
        if experiments := (node.get("experiments") or []):
            for experiment in experiments:
                for rev in experiment.get("revs") or []:
                    if not rev.get("error"):
                        yield {"commit": commit, **parse_experiment(rev)}
        else:
            yield {"commit": commit, **parse_experiment(node)}


def load_experiments(json_filepath):
    with open(json_filepath, "r") as f:
        data = json.load(f)
    return list(parse_experiments(data))


In [6]:
import itertools
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn

In [7]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [8]:
def jprint(obj):
    print(json.dumps(obj, indent=2))

In [9]:
filepaths = list(Path(EXP_RESULT_DIR).glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
for exp in experiments:
    if 'mode' not in exp['params']['retriever']:
        exp['params']['retriever']['mode'] = 'new'
print(f"{len(experiments)} experiments")
jprint(next(iter(experiments), None))

8 experiments
{
  "commit": "workspace",
  "id": "workspace",
  "name": null,
  "params": {
    "dataset": {
      "path": "bdsaglam/musique-mini",
      "name": "answerable",
      "split": "validation"
    },
    "model": {
      "path": "bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-musique-merged",
      "temperature": 0.5,
      "top_p": 0.95,
      "few_shot_prob": 0.0
    },
    "retriever": {
      "name": "hybrid-tei",
      "top_k": 1,
      "mode": "new"
    },
    "repeat": 1,
    "run": 1,
    "devices": "1"
  },
  "metrics": {
    "exact_match": 0.29,
    "f1": 0.38924054112060663,
    "supporting.precision": 0.8011666666666667,
    "supporting.recall": 0.6169444444444444,
    "supporting.f1": 0.6783253968253967,
    "citation.precision": 0.3125,
    "citation.recall": 0.17416666666666666,
    "citation.f1": 0.21183333333333332
  }
}


In [10]:
df = pd.json_normalize(experiments)
if not df.empty:
    df.drop(columns=['params.devices'], inplace=True)
    mask = ~df['name'].isin(['regal-agas'])
    df = df[mask].copy()
    print(f"{len(df)} experiments before preprocessing")
    df.head()

8 experiments before preprocessing


In [11]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [12]:
if len(df):
    df['params.model.temperature'] = df['params.model.temperature'].astype(float).map(lambda x: round(x, 2))
    df['params.model.top_p'] = df['params.model.top_p'].astype(float).map(lambda x: round(x, 2))

In [13]:
if len(df):
    df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
    df.drop_duplicates(subset=param_cols, inplace=True, keep='last')

    print(f"{len(df)} experiments after preprocessing")
df.head()

4 experiments after preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.model.path,params.model.temperature,params.model.top_p,params.model.few_shot_prob,...,params.repeat,params.run,metrics.exact_match,metrics.f1,metrics.supporting.precision,metrics.supporting.recall,metrics.supporting.f1,metrics.citation.precision,metrics.citation.recall,metrics.citation.f1
2,09307a4e1a863884ddf6b8d4adf4648b46876701,b3f91ad1bd8fb5eaa8a4d56c58d118c2c5f579cb,unfit-cool,bdsaglam/musique-mini,answerable,validation,/home/baris/repos/verifiers/outputs/Llama-3.1-...,0.5,0.95,0.0,...,5,1,0.406667,0.499275,0.853056,0.746667,0.780786,0.678333,0.597500,0.625690
3,09307a4e1a863884ddf6b8d4adf4648b46876701,e9a4d24fca485e3b3c5d77ccc9927e60ba54d78d,mauve-barb,bdsaglam/musique-mini,answerable,validation,meta-llama/Llama-3.1-8B-Instruct,0.5,0.95,0.0,...,5,1,0.200000,0.293761,0.815333,0.642778,0.698656,0.362500,0.194722,0.241627
4,09307a4e1a863884ddf6b8d4adf4648b46876701,b44d4706fb38824aa5985c66eaa1cb73340af007,natal-tass,bdsaglam/musique-mini,answerable,validation,bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-mus...,0.5,0.95,0.0,...,5,1,0.386667,0.482194,0.805556,0.638333,0.694437,0.345833,0.199722,0.238802
7,e67498e21376c13b7baa7f42e3d7349f110caa7c,e67498e21376c13b7baa7f42e3d7349f110caa7c,None,bdsaglam/musique-mini,answerable,validation,bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-mus...,0.5,0.95,0.0,...,1,1,0.290000,0.389241,0.801167,0.616944,0.678325,0.312500,0.174167,0.211833


In [14]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/musique-mini']

- params.dataset.name: ['answerable']

- params.dataset.split: ['validation']

- params.model.path: ['/home/baris/repos/verifiers/outputs/Llama-3.1-8B-Instruct-ragent-grpo-20250520_080809-merged', 'meta-llama/Llama-3.1-8B-Instruct', 'bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-musique-merged']

- params.model.temperature: [0.5]

- params.model.top_p: [0.95]

- params.model.few_shot_prob: [0.0]

- params.retriever.name: ['hybrid-tei']

- params.retriever.top_k: [1]

- params.retriever.mode: ['new']

- params.repeat: [5, 1]

- params.run: [1]



In [15]:
df['params.repeat'] = df.apply(lambda row: row.get('params.repeat', 1), axis=1)
df.to_json('exps-new.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [16]:
df = pd.concat(
    [
        pd.read_json(filename, orient="records", lines=True)
        for filename in [
            "exps-old.jsonl",
            "exps-new.jsonl",
        ]
    ]
)
df['params.repeat'] = df.apply(lambda row: row.get('params.repeat', 1), axis=1)
df.to_json('exps.jsonl', orient='records', lines=True)

In [17]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [18]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [19]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

6 experiment configurations
['params.model.path', 'params.model.temperature', 'params.model.top_p', 'params.model.few_shot_prob', 'params.retriever.name', 'params.retriever.top_k', 'params.repeat', 'params.run', 'params.dataset.path', 'params.dataset.name', 'params.dataset.split']


In [20]:
def preprocess_config(config):
    return {k: round(v, 2) if isinstance(v, float) else v for k, v in config.items()}

In [21]:
if len(df):
    existing_configs = [preprocess_config(config) for config in df[target_params].to_dict(orient="records")]
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 243


In [22]:
# find the missing configurations
missing_configs = [
    preprocess_config(dict(kv))
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

2 missing configurations


In [23]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if "[" in str(arg_value):
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [24]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for i,exp_config in enumerate(missing_configs):
        command = make_command(exp_config)
        devices = DEVICE_CONFIGS[i % len(DEVICE_CONFIGS)]
        command += f" \\\n    -S devices='\"{devices}\"'"
        f.write(command)
        f.write("\n\n")

In [25]:
next(iter(existing_configs), None)

{'params.model.path': 'Qwen/Qwen2.5-Coder-32B-Instruct',
 'params.model.temperature': 0.5,
 'params.model.top_p': 0.95,
 'params.model.few_shot_prob': 0,
 'params.retriever.name': 'hybrid-tei',
 'params.retriever.top_k': 1,
 'params.repeat': nan,
 'params.run': 1,
 'params.dataset.path': 'bdsaglam/musique-mini',
 'params.dataset.name': 'answerable',
 'params.dataset.split': 'validation'}

In [26]:
next(iter(missing_configs), None)

{'params.dataset.name': 'answerable',
 'params.dataset.path': 'bdsaglam/musique-mini',
 'params.dataset.split': 'validation',
 'params.model.few_shot_prob': 0.0,
 'params.model.path': 'meta-llama/Llama-3.1-8B-Instruct',
 'params.model.temperature': 0.5,
 'params.model.top_p': 0.95,
 'params.repeat': 1,
 'params.retriever.name': 'hybrid-tei',
 'params.retriever.top_k': 1,
 'params.run': 1}